In [ ]:
# 🚀 PRODUCTION PROMOTION - NEW WORKFLOW (CONFIG-DRIVEN)
import mlflow
from mlflow.tracking import MlflowClient
import time
import yaml
import sys
import traceback
import requests
from typing import Optional, Dict, Tuple
from datetime import datetime
from pyspark.sql import SparkSession

print("=" * 80)
print("🚀 PRODUCTION PROMOTION (NEW WORKFLOW)")
print("=" * 80)

# LOAD PIPELINE CONFIGURATION

print("\n📋 Step 1: Loading configuration from pipeline_config.yml...")

try:
    import os

    config_path = "/Workspace/Repos/vipultak7171@gmail.com/ml-credit-risk/dev_env/pipeline_config.yml"

    if not os.path.exists(config_path):
        config_path = "/Workspace/ml-credit-risk/dev_env/pipeline_config.yml"
    
    with open(config_path, "r") as f:
        pipeline_cfg = yaml.safe_load(f)

    print(f"✅ Configuration loaded from: {config_path}")

except FileNotFoundError:
    print("❌ ERROR: pipeline_config.yml not found!")
    sys.exit(1)
except Exception as e:
    print(f"❌ ERROR loading configuration: {e}")
    traceback.print_exc()
    sys.exit(1)

# CONFIG CLASS

class Config:
    def __init__(self):
        MODEL_TYPE = pipeline_cfg["model"]["type"]
        UC_CATALOG = pipeline_cfg["model"]["catalog"]
        UC_SCHEMA = pipeline_cfg["model"]["schema"]
        BASE_NAME = pipeline_cfg["model"]["base_name"]
        
        self.MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.{BASE_NAME}_{MODEL_TYPE}"
        self.MODEL_TYPE = MODEL_TYPE
        
        self.STAGING_ALIAS = pipeline_cfg["aliases"]["staging"]
        self.PRODUCTION_ALIAS = pipeline_cfg["aliases"]["production"]
        
        self.PRIMARY_METRIC = pipeline_cfg["metrics"]["classification"]["primary_metric"]
        self.DIRECTION = pipeline_cfg["metrics"]["classification"]["direction"]
        
        self.UAT_ENABLED = pipeline_cfg["uat"]["enabled"]
        self.UAT_RESULTS_TABLE = pipeline_cfg["tables"]["uat_results"]

        self.SLACK_ENABLED = pipeline_cfg["notifications"]["enabled"]
        self.SLACK_WEBHOOK_URL = self._get_slack_webhook()
        
        self.TOLERANCE = 1e-6

        print(f"\n📊 Config Loaded → Model: {self.MODEL_NAME}, Slack Enabled: {self.SLACK_WEBHOOK_URL is not None}")

    def _get_slack_webhook(self) -> Optional[str]:
        """Slack Webhook Logic copied from v1 (working version)"""
        from pyspark.dbutils import DBUtils
        dbutils = DBUtils(spark)

        for scope in ["shared-scope", "dev-scope"]:
            try:
                webhook = dbutils.secrets.get(scope, "SLACK_WEBHOOK_URL")
                if webhook.strip():
                    print(f"✓ Slack webhook loaded from: {scope}")
                    return webhook
            except:
                pass
        print("⚠️ No Slack Webhook found")
        return None


config = Config()

print("=" * 80)

# SLACK NOTIFICATION (FROM v1 - WORKING BLOCK)

def send_slack_notification(message, level="info"):
    """Send Slack message using v1 logic"""

    if not config.SLACK_WEBHOOK_URL:
        print(f"⚠️ Slack is NOT configured. Message: {message}")
        return

    emoji_map = {"info": "ℹ️", "success": "✅", "warning": "⚠️", "error": "❌"}

    formatted_message = f"{emoji_map.get(level, 'ℹ️')} {message}"

    try:
        response = requests.post(
            config.SLACK_WEBHOOK_URL,
            json={"text": formatted_message},
            timeout=5
        )
        if response.status_code == 200:
            print("📨 Slack notification sent!")
        else:
            print(f"⚠ Slack response status: {response.status_code}")
    except Exception as e:
        print(f"⚠ Slack send error: {e}")

# MLFLOW INIT

print("\n🔧 Initializing MLflow...")

try:
    spark = SparkSession.builder.appName("ProductionPromotion").getOrCreate()
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()
    print("✅ MLflow Running")
except Exception as e:
    print(f"❌ Pipeline Init Failed: {e}")
    send_slack_notification(f"❌ Init Failed: {e}", "error")
    sys.exit(1)

# HELPER FUNCTIONS

def get_metric(run_id):
    try:
        return client.get_run(run_id).data.metrics.get(config.PRIMARY_METRIC)
    except:
        return None


# GET STAGING MODEL

def get_staging():
    print("\n📍 Fetching staging model...")
    try:
        mv = client.get_model_version_by_alias(config.MODEL_NAME, config.STAGING_ALIAS)
        return {"version": int(mv.version), "run": mv.run_id, "metric": get_metric(mv.run_id)}
    except:
        return None

# CHECK UAT

def check_uat(version):
    print("\n📍 Validating UAT results...")
    df = spark.table(config.UAT_RESULTS_TABLE).toPandas()

    res = df[df["model_version"].astype(str) == str(version)]
    if res.empty:
        return False, None

    latest = res.sort_values("timestamp").iloc[-1]
    return latest["uat_status"] == "PASSED", {
        "accuracy": latest["accuracy"],
        "precision": latest["precision"],
        "recall": latest["recall"],
        "f1": latest["f1"],
        "roc_auc": latest["roc_auc"]
    }


# =============================================
# PROMOTE TO PROD
# =============================================
def promote(model, uat):
    v = model["version"]

    print(f"\n🚀 Setting @{config.PRODUCTION_ALIAS} → v{v}")

    client.set_registered_model_alias(config.MODEL_NAME, config.PRODUCTION_ALIAS, v)

    send_slack_notification(
        f"🎉 Production Deployment SUCCESS\nModel: `{config.MODEL_NAME}`\nVersion: v{v}",
        "success"
    )

    print("\n🎉 MODEL LIVE IN PRODUCTION!")


# =============================================
# MAIN
# =============================================
def main():
    send_slack_notification("🔥 Promotion Pipeline Started")

    model = get_staging()
    if not model:
        send_slack_notification("❌ No staging model found", "error")
        sys.exit(1)

    ok, uat = check_uat(model["version"])
    if not ok:
        send_slack_notification("⚠️ UAT FAILED — Promotion Blocked", "warning")
        sys.exit(1)

    promote(model, uat)

    print("\n✨ Workflow Complete!")


if __name__ == "__main__":
    main()
